In [ ]:
#| export

"""
Created on Wed Mar 18 17:59:42 2020
@author: Sanguyu Xu
xusangyu@gmail.com
"""
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import locoDataMunger
import locoUtilities
import locoPlotters
import plotTools as pt
# import pickleo
import dabest
from plotTools import setFont


In [ ]:
#| export
class EspressoLocomotion(object):
    def __init__(self, dataFolder, startMin, endMin, companionEspObj=None, initialResamplePeriod=50, smoothing=True, longForm=False):
        self.version = '0.1.5'
        if dataFolder[-1] != '/':
            dataFolder = dataFolder+'/'
        self.metaDataDf, self.countLogDf, self.portLocationsDf, self.experimentSummary = locoDataMunger.readMetaAndCount(
            dataFolder, companionEspObj, startMin, endMin, initialResamplePeriod, smoothing, longForm)
        outputDir = locoUtilities.makeOutputFolders(dataFolder)
        self.dataFolder = dataFolder
        self.outputFolder = outputDir
        self.startMin = startMin
        self.endMin = endMin
        self.resultsDf = self.metaDataDf
        self.resultsDf['averageSpeed_mm/s'] = np.nanmean(
            self.countLogDf.filter(regex='_V'), axis=0)
        self.resultsDf['xPosition_mm'] = np.nanmean(
            self.countLogDf.filter(regex='_X'), axis=0)
        self.resultsDf['yPosition_mm'] = np.nanmean(
            self.countLogDf.filter(regex='_Y'), axis=0)
        self.resultsDf['inLeftPort'] = np.nanmean(
            100*(self.countLogDf.filter(regex='LeftPort') > 0), axis=0)
        self.resultsDf['inRightPort'] = np.nanmean(
            100*(self.countLogDf.filter(regex='RightPort') > 0), axis=0)
        if any(self.experimentSummary.countLogDate.str.contains(self.resultsDf.Date[0])):
            self.resultsDf['countLogDate'] = self.resultsDf['Date']
            self.resultsDf['feedLogDate'] = [self.experimentSummary.loc[self.experimentSummary['countLogDate']
                                                                        == d]['feedLogDate'].iloc[0] for d in self.resultsDf['Date']]
        else:
            self.resultsDf['feedLogDate'] = self.resultsDf['Date']
            self.resultsDf['countLogDate'] = [self.experimentSummary.loc[self.experimentSummary['feedLogDate']
                                                                         == d]['countLogDate'].iloc[0] for d in self.resultsDf['Date']]
    def versionNotes(self):
        print('version notes 0.1.1: added portlocations')
        print('version notes 0.1.2: added gaussian smoothing of x y locations')
        print('version notes 0.1.3: added detection of fall events')
        print('version notes 0.1.4: bugfix setfont and moved colorbar on heatmap to bottom')
        print('version notes 0.1.5: added handling of dabest 0.3.9999')
        print('version notes 0.2.0: major refactoring')
 
    def plotChamberSmallMultiples(self, ncol=15, customPalette=None, setNumber=None):
        dates = self.metaDataDf.Date.unique()
        chamberSmallsTrack = np.empty(len(dates), dtype=object)
        axarrT = np.empty(len(dates), dtype=object)
        chamberSmallsHeat = np.empty(len(dates), dtype=object)
        axarrH = np.empty(len(dates), dtype=object)
        print('Espresso Runs found:\n')
        print(dates)
        if setNumber is not None:
            dates = [dates[setNumber]]
        for i in range(0, len(dates)):
            print('\n\n plotting ' + dates[i] + '...')
            portFile = self.experimentSummary.loc[self.experimentSummary['metaDataFile']
                                                  == 'MetaData_'+dates[i] + '.csv']['portLocationsFile'].iloc[0]
            portDate = locoDataMunger.extractDateStr(portFile)[0]
            submeta = self.metaDataDf.loc[self.metaDataDf.Date == dates[i]]
            subcount = self.countLogDf.filter(regex=dates[i])
            subport = self.portLocationsDf.loc[self.portLocationsDf.Date == portDate]
            chamberSmallsTrack[i], axarrT[i] = locoPlotters.putThingsInToChamberSubplot(
                subcount, submeta, subport, customPalette, ncol)
            chamberSmallsHeat[i], axarrH[i] = locoPlotters.putThingsInToChamberSubplot(
                subcount, submeta, subport, customPalette, ncol, locoPlotters.espressoPlotHeatmap)
            outputDir = self.outputFolder + 'chamberPlots/'
            locoUtilities.espressoSaveFig(
                chamberSmallsTrack[i], 'chamberSmallsTrack', dates[i], outputDir, pngDPI=200)
            locoUtilities.espressoSaveFig(
                chamberSmallsHeat[i], 'chamberSmallsHeat', dates[i], outputDir, pngDPI=200)
        self.chamberSmallsTrack = chamberSmallsTrack
        self.axarrT = axarrT
        self.chamberSmallsHeat = chamberSmallsHeat
        self.axarrH = axarrH
        return chamberSmallsTrack, axarrT, chamberSmallsHeat, axarrH

    def plotMeanHeatMaps(self, binSize=0.2, row=None, col=None, reverseRows=False, reverseCols=False,  verbose=False, heatmapCMap='RdYlBu_r', smooth=2):
        heatMapOutputDir = self.outputFolder
        if verbose:
            meanHeatmapFig,  Hall, images, smallHeatmapFigs = locoPlotters.espressoPlotMeanHeatmaps(
                self, binSize, None, None, False, False, verbose, heatmapCMap, smooth)
            locoUtilities.espressoSaveFig(
                smallHeatmapFigs, 'smallHeatmapFigs', self.metaDataDf.Date[0], heatMapOutputDir, pngDPI=200)
        else:
            meanHeatmapFig, Hall, images = locoPlotters.espressoPlotMeanHeatmaps(
                self, binSize, row, col, reverseRows, reverseCols, verbose, heatmapCMap, smooth)
        # self.resultsDf = resultsDf
        self.heatmapMatrix = Hall
        self.meanHeatmapFig = meanHeatmapFig
        self.heatmapImages = images

    def plotBoundedLines(self,  colorBy, locoSuffix='V', row=None, col=None, rp='300s', YLim=[], customPalette={}, reverseRows=False, reverseCol=False, xUnit='min'):
        if xUnit == 'hour':
            T = self.countLogDf.iloc[:, 0]/3600
        else:
            T = self.countLogDf.iloc[:, 0]/60

        if 'Port' in locoSuffix:
            metric = 100*(self.countLogDf.filter(regex='_' + locoSuffix) > 0)
        else:
            metric = self.countLogDf.filter(regex='_' + locoSuffix)

        listOfPlots, gp, cPalette = locoPlotters.subplotRowColColor(
            self.metaDataDf, colorBy, row, col, reverseRows, reverseCol)
        if customPalette:
            cPalette = customPalette
        nr, nc = listOfPlots[-1][0][0:2]
        figure, axes = plt.subplots(
            nrows=nr + 1, ncols=nc + 1, squeeze=False, figsize=(3 * (nc + 1), 3 * (nr + 1)))
        plotNames = [0]*len(listOfPlots)
        meanLines = [0]*len(listOfPlots)
        ciBounds = [0]*len(listOfPlots)

        yl = {'V': 'Average Speed (mm/s)', 'X': 'X Position (mm)', 'Y': 'Y Position (mm)',
              'InLeftPort': 'Percent time in Left Port',  'InRightPort': 'Percent time in Right Port'}
        for i in range(0, len(listOfPlots)):
            # print(listOfPlots[i])
            ro, co = listOfPlots[i][0][0:2]
            name = listOfPlots[i][1]
            ind = gp[name]
            locoPlotters.plotBoundedLine(
                T, metric.iloc[:, ind], ax=axes[ro, co], c=cPalette[name[-1]], resamplePeriod=rp)
            meanLines[i] = axes[ro, co].meanLine[0]
            ciBounds[i] = axes[ro, co].ciBound
            if co == 0:
                axes[ro, co].set_ylabel(yl[locoSuffix])
            if ro == axes.shape[0]:
                axes[ro, co].set_xlabel('Time (hour)')
            axes[ro, co].set_title(name[0] + ' ' + name[1])
            axes[ro, co]. spines["right"].set_visible(False)
            axes[ro, co]. spines["top"].set_visible(False)
            plotNames[i] = name[-1]
        ylims = [ax.get_ylim() for ax in axes.flatten()]
        for i in range(0, len(listOfPlots)):
            ro, co = listOfPlots[i][0][0:2]
            if len(YLim) > 0:
                axes[ro, co].set_ylim(YLim)
            else:
                axes[ro, co].set_ylim([np.min(ylims), np.max(ylims)])
            locoPlotters.setAxesTicks(axes[ro, co], True, gridState=False)

        axes[ro, co].legend(plotNames, loc='upper right')

        locoUtilities.espressoSaveFig(
            figure, 'boundedTS_' + locoSuffix + '_', self.metaDataDf.Date[0], self.outputFolder)

        return figure, axes, meanLines, ciBounds

    def calculatePeriFeedSpeed(self, companionEspObj, monitorWindow=120, startSeconds=0):

        self.feedsRevisedDf, self.countLogDf, self.meanPeriSpeed, self.maxSpeed= locoDataMunger.calculatePeriFeedLoco(
            self.countLogDf, self.portLocationsDf, companionEspObj, self.experimentSummary, monitorWindow, startSeconds)
        self.resultsDf['ChamberID'] = self.resultsDf['feedLogDate'] + \
            '_Chamber' + self.resultsDf['ID'].astype(str)
        print(self.resultsDf.columns)
        print(self.meanPeriSpeed.columns)
        if str(monitorWindow)+'beforeFeedSpeed_mm/s_Mean' in self.resultsDf.columns:
            self.resultsDf[['ChamberID', str(monitorWindow)+'beforeFeedSpeed_mm/s_Mean', 'duringFeedSpeed_mm/s_Mean',  str( 
                monitorWindow)+'afterFeedSpeed_mm/s_Mean']] = self.meanPeriSpeed[['ChamberID', str(monitorWindow)+'beforeFeedSpeed_mm/s_Mean', 'duringFeedSpeed_mm/s_Mean',  str( 
                monitorWindow)+'afterFeedSpeed_mm/s_Mean']]
        else:
            self.resultsDf = pd.merge(
                self.meanPeriSpeed, self.resultsDf, how="outer", on='ChamberID')
        for c in self.resultsDf.columns:
            if 'mm/s' not in c:
                self.resultsDf[c].fillna(0, inplace=True)

        self.monitorMin = str(int(monitorWindow/60))+' min'
        self.outputPrefix = self.outputFolder+self.monitorMin
        
        PeriFeedDiagonal = locoPlotters.plotPeriFeedDiagonal(self, monitorWindow)
        pairedSpeedPlots = locoPlotters.plotPairedSpeeds(self, monitorWindow)
        
    def calculateFallEvents(self, nstd=4, windowsize=1000, ewm1=12, ewm2=26, ewm3=9):
        # added Jan 2022 to detect falls
        print('Detecting Fall Events...')
        falls, newCountLog = locoDataMunger.fallEvents(
            self.countLogDf, nstd, windowsize, ewm1, ewm2, ewm3)
        self.resultsDf['falls'] = np.nanmax(falls, axis=0)
        self.countLogDf = newCountLog
        print('Done')